In [10]:
import os
import warnings

# from vivit import ViViT

import numpy as np
from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torchvision.io as io # for reading video files
import torchvision.transforms as transforms

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.backends.cudnn.enabled = True #Enable cuDNN
    torch.backends.cudnn.benchmark = True #Enable cuDNN benchmark for the best performance
    warnings.filterwarnings("ignore") #I realy don't care that you broke your elbow

print("Using device: ", device)

Using device:  cpu


In [13]:
import torch
from torch import nn, einsum
import torch.nn.functional as F
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
# from module import Attention, PreNorm, FeedForward
import numpy as np

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        self.norm = nn.LayerNorm(dim)
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return self.norm(x)


  
class ViViT(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, num_frames, dim = 192, depth = 4, heads = 3, pool = 'cls', in_channels = 3, dim_head = 64, dropout = 0.,
                 emb_dropout = 0., scale_dim = 4, ):
        super().__init__()
        
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'


        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = in_channels * patch_size ** 2
        self.to_patch_embedding = nn.Sequential(
            Rearrange('b t c (h p1) (w p2) -> b t (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_frames, num_patches + 1, dim))
        self.space_token = nn.Parameter(torch.randn(1, 1, dim))
        self.space_transformer = Transformer(dim, depth, heads, dim_head, dim*scale_dim, dropout)

        self.temporal_token = nn.Parameter(torch.randn(1, 1, dim))
        self.temporal_transformer = Transformer(dim, depth, heads, dim_head, dim*scale_dim, dropout)

        self.dropout = nn.Dropout(emb_dropout)
        self.pool = pool

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        x = self.to_patch_embedding(x)
        b, t, n, _ = x.shape

        cls_space_tokens = repeat(self.space_token, '() n d -> b t n d', b = b, t=t)
        x = torch.cat((cls_space_tokens, x), dim=2)
        x += self.pos_embedding[:, :, :(n + 1)]
        x = self.dropout(x)

        x = rearrange(x, 'b t n d -> (b t) n d')
        x = self.space_transformer(x)
        x = rearrange(x[:, 0], '(b t) ... -> b t ...', b=b)

        cls_temporal_tokens = repeat(self.temporal_token, '() n d -> b n d', b=b)
        x = torch.cat((cls_temporal_tokens, x), dim=1)

        x = self.temporal_transformer(x)
        

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        return self.mlp_head(x)
    
    
    

if __name__ == "__main__":
    
    img = torch.ones([1, 16, 3, 224, 224]).cuda()
    
    model = ViViT(224, 16, 100, 16).cuda()
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    parameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000
    print('Trainable Parameters: %.3fM' % parameters)
    
    out = model(img)
    
    print("Shape of out :", out.shape)      # [B, num_classes]

    
    

ModuleNotFoundError: No module named 'module'

In [ ]:
params = {
    "batch_size": 1,
    "epochs": 50,
    "lr": 0.001,
    "momentum": 0.9,
    "log_interval": 10,
    # "pin_memory": True,
    "patience": 20,
    "frame_size": (256,256),
    "vid_length": 70,
}

# Data Preprocessing

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, transform=None):
        self.root_dir = "data/videos/"
        # self.root_dir = "C:\\Users\\Carter\\Desktop\\Testing Data\\videos_new\\"
        # Get the list of all files in directory that end in .mp4
        self.files = [f for f in os.listdir(self.root_dir) if f.endswith('.mp4')]

        self.transform = transform

        self.data = []

        for file in tqdm(self.files):
        # for file in self.files:
            # Read the video file
            pin = [int(x) for x in file[0:4]] # Get the pin from the filename (first 4 characters)
            # pin = torch.tensor(pin, dtype=torch.float)
            # Append the video to the data
            self.data.append((file, pin))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get the video and pin
        pin = self.data[idx][1]

        #For now, only return the first digit of the pin
        pin = pin[0]

        #One hot encode the pin
        pin = np.eye(10)[pin]
        # print(f'~~~~ PIN:{pin}')

        pin = torch.tensor(pin, dtype=torch.float)

        # Read the video file
        video_path = self.root_dir + self.data[idx][0]
        video, audio, info = io.read_video(video_path, pts_unit='sec', output_format='TCHW') #Get in time, channel, hieght width format

        #Apply the transform to the video
        if self.transform:
            video = self.transform(video)[0:params["vid_length"]] #Only take the first 70 frames

        #If the video is shorter than 70 frames, print the filename
        if video.shape[0] < params["vid_length"]:
            print("Short video: ", self.data[idx][0])

        #Convert the video to a tensor
        video = torch.tensor(video, dtype=torch.float)    

        # Return the video and the pin
        return video, pin


# Create the datasets
dataset = CustomDataSet(
    transform=transforms.Compose([
        transforms.Resize(params["frame_size"]),
    ])
)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

print("Train size: ", train_size)
print("Test size: ", test_size)

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

100%|██████████| 929/929 [00:00<00:00, 929288.91it/s]

Train size:  743
Test size:  186


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=True, pin_memory=True)

# Training and Testing Loop


In [ ]:
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    total_correct = 0

    for i, x in tqdm(enumerate(dataloader), desc="Training", leave=False):
        videos = x[0].to(device, non_blocking=True)
        labels = x[1].to(device, non_blocking=True)

        # optimizer.zero_grad()
        #This is a bit faster than the above line
        for param in model.parameters():
            param.grad = None

        # Forward pass
        outputs = model(videos)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        total_correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    return avg_loss

In [ ]:
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0

    with torch.no_grad():
        for i, x in tqdm(enumerate(dataloader), desc="Validation", leave=False):
            
            videos = x[0].to(device, non_blocking=True)
            labels = x[1].to(device, non_blocking=True)

            outputs = model(videos)

            #Print the output and label
            # for i in range(len(outputs)):
                # print(outputs[i], labels[i])

            loss = criterion(outputs, labels)

            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [ ]:
def run_epoch(model, train_loader, val_loader, criterion, optimizer, history):
    best_val_loss = float("inf")

    for epoch in range(history["last_epoch"] + 1, params["epochs"] + 1):
        train_loss = train(
            model,
            train_loader,
            criterion,
            optimizer,
        )
        val_loss = validate(model, val_loader, criterion)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            history["best_weights"] = model.state_dict()
            patience = 0
        else:
            patience += 1

        if patience >= params["patience"]:
            print(f"Early stopping at epoch {epoch}")
            break

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["last_epoch"] = epoch

        print(f'Epoch:              {epoch}/{params["epochs"]}')
        print(f"Training Loss:      {train_loss:.4f}")
        print(f"Validation Loss:    {val_loss:.4f}")
        print(f'Learning rate:      {optimizer.param_groups[0]["lr"]:.7f}')

    return (model, history)

In [ ]:
model = ViViT(image_size=params["frame_size"][0], patch_size=16, num_classes=10, num_frames=70).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

patience = 0

best_val_loss = float("inf")
besst_weights = None

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

try:
    model, history = run_epoch(
        model,
        train_loader,
        test_loader,
        criterion,
        optimizer,
        history={
            "train_loss": [],
            "val_loss": [],
            "val_accuracy": [],
            "last_epoch": 0,
            "best_weights": None,
        },
    )

except KeyboardInterrupt:
    print("Training interrupted")

Training: 0it [00:00, ?it/s]

~~~~ PIN:[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


Training: 1it [00:00,  2.11it/s]

~~~~ PIN:[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


Training: 2it [00:00,  2.53it/s]

...

Training: 252it [01:16,  3.47it/s]

~~~~ PIN:[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


Training: 253it [01:17,  3.60it/s]

~~~~ PIN:[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


~~~~ PIN:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Training interrupted
